## Batch Deploymnet

Most deployment use cases don't require real-time inference. In batch deployment:

1. A batch endpoint is provisioned
2. A compute cluster is provisioned
3. Input data path is provided

The batch deployment can be scheduled via ADF or event grid. For testing one can invoke a batch deployment using azure ml sdk

In [1]:
from azure.ai.ml import MLClient
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential

### Get Credentials

In [2]:
keyVaultName = "mobilepricinga6945442583"
KVUri = f"https://{keyVaultName}.vault.azure.net"
credential = DefaultAzureCredential()
client = SecretClient(vault_url=KVUri, credential=credential)

In [3]:
subs_id = client.get_secret("subscription-id").value
rg_name = client.get_secret("ml-resource-group").value
ws_name = client.get_secret("ml-workspace-name").value

### Create Client

In [4]:
ml_client = MLClient(
    credential=credential,
    subscription_id=subs_id,
    resource_group_name=rg_name,
    workspace_name=ws_name,
)

In [5]:
#ml_client.models.download(name="mobile_pricing_model",version="2",download_path=".")

### Use the model from workspace

In [6]:
model = ml_client.models.get(name="mobile_pricing_model",version="2")

### Create compute infrastructure

In [7]:
from azure.ai.ml.entities import BatchEndpoint, ModelBatchDeployment, ModelBatchDeploymentSettings, PipelineComponentBatchDeployment, Model, AmlCompute, Data, BatchRetrySettings, CodeConfiguration, Environment, Data

In [9]:
compute_name = "batch-cluster"
if not any(filter(lambda m: m.name == compute_name, ml_client.compute.list())):
    compute_cluster = AmlCompute(
        name=compute_name, description="amlcompute", min_instances=0, max_instances=2
    )
    ml_client.begin_create_or_update(compute_cluster).result()

### Create Endpoint

In [10]:
endpoint_name = "mobile-pricing-batch-endpoint"
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A mobile pricing batch inference classifier",
)

In [11]:
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

BatchEndpoint({'scoring_uri': 'https://mobile-pricing-batch-endpoint.eastus.inference.ml.azure.com/jobs', 'openapi_uri': None, 'provisioning_state': 'Succeeded', 'name': 'mobile-pricing-batch-endpoint', 'description': 'A mobile pricing batch inference classifier', 'tags': {}, 'properties': {'BatchEndpointCreationApiVersion': '2022-05-01', 'azureml.onlineendpointid': '/subscriptions/0aa1c63a-7a46-403c-91e4-8ec91264bc42/resourceGroups/rg-mobile-pricing-dev/providers/Microsoft.MachineLearningServices/workspaces/mobile-pricing-azml-dev/batchEndpoints/mobile-pricing-batch-endpoint'}, 'print_as_yaml': True, 'id': '/subscriptions/0aa1c63a-7a46-403c-91e4-8ec91264bc42/resourceGroups/rg-mobile-pricing-dev/providers/Microsoft.MachineLearningServices/workspaces/mobile-pricing-azml-dev/batchEndpoints/mobile-pricing-batch-endpoint', 'Resource__source_path': None, 'base_path': '/Users/gunnvantsaini/Library/CloudStorage/OneDrive-Personal/consulting/setu/mlops_bitbucket/content/01_Mlops_Using_Cloud_Too